In [1]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

! pip install xgboost


import xgboost as xgb

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_b513d501b0314f9ca0a4b43814f413b8 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_b513d501b0314f9ca0a4b43814f413b8 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_b513d501b0314f9ca0a4b43814f413b8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QzyMwuqEgykPxvDFxI8Fs1uE1jjFlchMleAAvNmFK07q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_b513d501b0314f9ca0a4b43814f413b8)

body = client_b513d501b0314f9ca0a4b43814f413b8.get_object(Bucket='coursemlzoomcamp-donotdelete-pr-u0vglgqkbod79i',Key='Churn_Modelling.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df_data_1

In [4]:
df.columns = df.columns.str.lower()

objects = list(df.dtypes[df.dtypes == "object"].index)

for col in objects:
    df[col] = df[col].str.lower()


In [5]:
del df["rownumber"]
del df["surname"]

In [6]:
df_full_train, df_test = train_test_split(df , test_size = 0.2, random_state = 1 )
df_train , df_val = train_test_split(df_full_train,test_size = 0.25, random_state = 11)

y_train  = df_train.exited.values
y_val = df_val.exited.values
y_test= df_test.exited.values

del df_train["exited"]
del df_val["exited"]
del df_test["exited"]

len(df_train),len(df_val),len(df_test)

(6000, 2000, 2000)

In [7]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.to_dict(orient = 'records' )
x_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient = 'records' )
x_val = dv.transform(val_dicts)

test_dicts = df_test.to_dict(orient = 'records')
x_test = dv.transform(test_dicts)

In [8]:
fetures = dv.get_feature_names()

dtrain = xgb.DMatrix( x_train , label = y_train , feature_names = fetures)
dval = xgb.DMatrix( x_val , label = y_val , feature_names = fetures)
dtest= xgb.DMatrix(x_test, label = y_test , feature_names = fetures)

In [9]:
xgb_params = {
    'eta':0.1, 
    'max_depth': 3,
    'min_child_weight': 10,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=60,
                  verbose_eval=5,
                  )

In [10]:
y_pred = model.predict(dval)
roc_auc_score(y_val, y_pred)


0.8576825004955579

In [11]:
y_pred = model.predict(dtest)
roc_auc_score(y_test, y_pred)

0.8789023602295618

In [12]:
import pickle

In [13]:
with open("model_pickle","wb")as f_out:
    pickle.dump((dv,model),f_out)

### *Why isn't the file downloading on my computer?*


In [14]:
with open("model_pickle","rb")as f_in:
    dv, model = pickle.load(f_in)

In [15]:
df.iloc[0].to_dict()

{'customerid': 15634602,
 'creditscore': 619,
 'geography': 'france',
 'gender': 'female',
 'age': 42,
 'tenure': 2,
 'balance': 0.0,
 'numofproducts': 1,
 'hascrcard': 1,
 'isactivemember': 1,
 'estimatedsalary': 101348.88,
 'exited': 1}

In [16]:
customer = df.iloc[0].to_dict()

In [17]:
x = dv.transform(customer)
dx = xgb.DMatrix(x,feature_names = fetures)

In [18]:
model.predict(dx)

array([0.2655728], dtype=float32)

Restarting the kernel and loading the model


In [1]:
import pickle

In [2]:
import xgboost as xgb

In [2]:
with open("model_pickle","rb")as f_in:
    dv, model = pickle.load(f_in)

In [3]:
customer = {'customerid': 15634602,
 'creditscore': 619,
 'geography': 'france',
 'gender': 'female',
 'age': 42,
 'tenure': 2,
 'balance': 0.0,
 'numofproducts': 1,
 'hascrcard': 1,
 'isactivemember': 1,
 'estimatedsalary': 101348.88
           }

In [4]:
fetures = ['age',
 'balance',
 'creditscore',
 'customerid',
 'estimatedsalary',
 'gender=female',
 'gender=male',
 'geography=france',
 'geography=germany',
 'geography=spain',
 'hascrcard',
 'isactivemember',
 'numofproducts',
 'tenure']

In [8]:
x = dv.transform(customer)
dx = xgb.DMatrix(x,feature_names = fetures)

In [10]:
dx

In [11]:
model.predict(dx)

array([0.2655728], dtype=float32)